# Can sentiment be predicted on French Twitter data?

Loading needed python packages

In [24]:
import csv
import pandas as pd 
import numpy as np  
import re  
import nltk  
from sklearn.datasets import load_files  
nltk.download('stopwords')  
import pickle  
from nltk.corpus import stopwords
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, numpy, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from keras.preprocessing import text
import gensim 
from gensim.models import Word2Vec
import matplotlib.pyplot as plt

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


Importing the data

The comment data is spread over 2 collums, because the data contains also ´,´ which is used as delimter value.

In [ ]:
df = pd.read_csv("/home/sije/Thesis/data/French-Sentiment-Analysis-Dataset-master/tweets.csv", delimiter = ",",error_bad_lines=False)

In [ ]:
df['comment'] = df[['comment1', 'comment2']].apply(lambda x: ''.join(str(x)), axis=1)

Rows which have missing values are removed

In [27]:
df = df.dropna()

The dataset consits out two labels and the numbers are almost equal.

In [28]:
df['Label'].value_counts()

0          43306
4          38355
L'homme        1
Name: Label, dtype: int64

This stemming function is applies a French steming on the data. Also unwanted data is removed

In [10]:
documents = []

from nltk.stem.snowball import FrenchStemmer

stemmer = FrenchStemmer()
def get_stemming(sen):  
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(sen))

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    document = document.split()

    document = [stemmer.stem(word) for word in document]
    document = ' '.join(document)

    return(document)

In [11]:
df['message_stemmer'] = df['comment'].apply(get_stemming)

Dataset is split in training and validation set

In [12]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(df['message_stemmer'], df['Label'])

Label are created and the target variable are encode

In [13]:
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

3 data transformations are created word level tf-idf, ngram level tf-idf and characters level tf-idf

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['message_stemmer'])
xtrain_tfidf =  tfidf_vect.transform(X_train.apply(lambda x: np.str_(x)))
xvalid_tfidf =  tfidf_vect.transform(X_test.apply(lambda x: np.str_(x)))

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df['message_stemmer'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(X_train.apply(lambda x: np.str_(x)))
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(X_test.apply(lambda x: np.str_(x)))

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(df['message_stemmer'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(X_train.apply(lambda x: np.str_(x)))
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(X_test.apply(lambda x: np.str_(x)))

In [14]:
count_vect = CountVectorizer()
count_vect.fit(df['message_stemmer'])

xtrain_count =  count_vect.transform(X_train.apply(lambda x: np.str_(x)))
xvalid_count =  count_vect.transform(X_test.apply(lambda x: np.str_(x)))

Model function which will be used for all different models, which returns a confusion matrix, classifaction report and accuracy

In [16]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    print(confusion_matrix(y_test,predictions))  
    print(classification_report(y_test,predictions))  
    print(accuracy_score(y_test,predictions))

In [17]:
# Naive Bayes on Count Vectors
print("NB, Count Vectors: ")
train_model(naive_bayes.MultinomialNB(), xtrain_count, y_train, xvalid_count)


# Naive Bayes on Word Level TF IDF Vectors
print("NB, WordLevel TF-IDF: ")
train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, y_train, xvalid_tfidf)

# Naive Bayes on Ngram Level TF IDF Vectors
print("NB, N-Gram Vectors: ")
train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, y_train, xvalid_tfidf_ngram)


# Naive Bayes on Character Level TF IDF Vectors
print("NB, CharLevel Vectors: ")
train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, y_train, xvalid_tfidf_ngram_chars)

NB, Count Vectors: 
[[8139 2644]
 [4287 5346]]
              precision    recall  f1-score   support

           0       0.65      0.75      0.70     10783
           1       0.67      0.55      0.61      9633

   micro avg       0.66      0.66      0.66     20416
   macro avg       0.66      0.65      0.65     20416
weighted avg       0.66      0.66      0.66     20416

0.6605113636363636
NB, WordLevel TF-IDF: 
[[8030 2753]
 [4279 5354]]
              precision    recall  f1-score   support

           0       0.65      0.74      0.70     10783
           1       0.66      0.56      0.60      9633

   micro avg       0.66      0.66      0.66     20416
   macro avg       0.66      0.65      0.65     20416
weighted avg       0.66      0.66      0.65     20416

0.655564263322884
NB, N-Gram Vectors: 
[[7465 3318]
 [3889 5744]]
              precision    recall  f1-score   support

           0       0.66      0.69      0.67     10783
           1       0.63      0.60      0.61      9633



In [18]:
# Linear Classifier on Count Vectors
print("LR, Count Vectors: ")
train_model(linear_model.LogisticRegression(), xtrain_count, y_train, xvalid_count)


# Linear Classifier on Word Level TF IDF Vectors
print("LR, WordLevel TF-IDF: ")
train_model(linear_model.LogisticRegression(), xtrain_tfidf, y_train, xvalid_tfidf)


# Linear Classifier on Ngram Level TF IDF Vectors
print("LR, N-Gram Vectors: ")
train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, y_train, xvalid_tfidf_ngram)


# Linear Classifier on Character Level TF IDF Vectors
print("LR, CharLevel Vectors: ")
train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, y_train, xvalid_tfidf_ngram_chars)


LR, Count Vectors: 


/home/sije/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/sije/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[[7421 3362]
 [3525 6108]]
              precision    recall  f1-score   support

           0       0.68      0.69      0.68     10783
           1       0.64      0.63      0.64      9633

   micro avg       0.66      0.66      0.66     20416
   macro avg       0.66      0.66      0.66     20416
weighted avg       0.66      0.66      0.66     20416

0.6626665360501567
LR, WordLevel TF-IDF: 
[[7555 3228]
 [3597 6036]]
              precision    recall  f1-score   support

           0       0.68      0.70      0.69     10783
           1       0.65      0.63      0.64      9633

   micro avg       0.67      0.67      0.67     20416
   macro avg       0.66      0.66      0.66     20416
weighted avg       0.67      0.67      0.67     20416

0.6657033699059561
LR, N-Gram Vectors: 
[[7435 3348]
 [3847 5786]]
              precision    recall  f1-score   support

           0       0.66      0.69      0.67     10783
           1       0.63      0.60      0.62      9633

   micro avg       

In [19]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, y_train, xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors: " + str(accuracy))

/home/sije/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[[10783     0]
 [ 9633     0]]
              precision    recall  f1-score   support

           0       0.53      1.00      0.69     10783
           1       0.00      0.00      0.00      9633

   micro avg       0.53      0.53      0.53     20416
   macro avg       0.26      0.50      0.35     20416
weighted avg       0.28      0.53      0.37     20416

0.528164184952978
SVM, N-Gram Vectors: None


/home/sije/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, y_train, xvalid_count)
print("RF, Count Vectors: " + str(accuracy))

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, y_train, xvalid_tfidf)
print("RF, WordLevel TF-IDF: " + str(accuracy))

/home/sije/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[[8175 2608]
 [5123 4510]]
              precision    recall  f1-score   support

           0       0.61      0.76      0.68     10783
           1       0.63      0.47      0.54      9633

   micro avg       0.62      0.62      0.62     20416
   macro avg       0.62      0.61      0.61     20416
weighted avg       0.62      0.62      0.61     20416

0.6213264106583072
RF, Count Vectors: None


/home/sije/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[[8077 2706]
 [5128 4505]]
              precision    recall  f1-score   support

           0       0.61      0.75      0.67     10783
           1       0.62      0.47      0.53      9633

   micro avg       0.62      0.62      0.62     20416
   macro avg       0.62      0.61      0.60     20416
weighted avg       0.62      0.62      0.61     20416

0.6162813479623824
RF, WordLevel TF-IDF: None


The Linear classifcation using CharLevel Vectors results in an accuracy 0.95. Therefore it is the best scoring model. With an accurancy of 0.95 can be said that a sentiment model can be made for French twitter data.